In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'  
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

In [ ]:
from proj1_utils import *
from proj1_visualization import *
from proj1_cross_validation import *
from implementations import *

In [ ]:
cat_cols = [22]
tX_num, tX_cat = split_numerical_categorical(tX,cat_cols)
tX_test_num, tX_test_cat = split_numerical_categorical(tX_test,cat_cols)

## Preprocessing

In [ ]:
# Treat numerical values
best_degree = 5
full_x_train_num_nan = replace_undef_val_with_nan(tX_num)
full_x_train_num_std, train_mean, train_std = nan_standardize_fit(full_x_train_num_nan)
full_x_train_num_valid = replace_nan_val_with_median(full_x_train_num_std)
full_x_train_num_valid = replace_iqr_outliers(full_x_train_num_valid)
# Treat categorical values
full_x_train_ohe_cat = one_hot_encode(tX_cat)
full_x_train_poly = build_poly(full_x_train_num_valid , best_degree)
full_x_train = np.hstack((full_x_train_poly,full_x_train_ohe_cat))
# Treat labels
full_y_train = y

In [ ]:
x_train, y_train, x_val, y_val = split_data(full_x_train,full_y_train,0.8)

In [ ]:
# Treat numerical values
x_test_num_nan = replace_undef_val_with_nan(tX_test_num)
x_test_num_nan_std = nan_standardize_transform(x_test_num_nan,train_mean,train_std)
x_test_num_valid_std = replace_nan_val_with_median(x_test_num_nan_std)
x_test_num_valid_std = replace_iqr_outliers(x_test_num_valid_std)
# Treat categorical values
x_test_ohe_cat = one_hot_encode(tX_test_cat)
x_test_poly = build_poly(x_test_num_valid_std , best_degree)
x_test = np.hstack((x_test_poly,x_test_ohe_cat))

## Test grid search

### Least squares GD

In [ ]:
method_flag = 1
degrees = np.asarray(range(1,4))
lambdas = np.asarray([0])
best_degree, best_lambda, accuracy_score, accuracy_scores_grid = degree_lambda_grid_search(y,tX,[22],0.8,method_flag,degrees,lambdas)
print(best_degree)
print(best_lambda)
print(accuracy_score)
print(accuracy_scores_grid)

### Least squares SGD

In [ ]:
method_flag = 2
degrees = np.asarray(range(1,6))
lambdas = np.asarray([0])
best_degree, best_lambda, accuracy_score, accuracy_scores_grid = degree_lambda_grid_search(y,tX,[22],0.8,method_flag,degrees,lambdas)
print(best_degree)
print(best_lambda)
print(accuracy_score)
print(accuracy_scores_grid)

### Least squares

In [ ]:
method_flag = 3
degrees = np.asarray(range(1,11))
lambdas = np.asarray([0])
best_degree, best_lambda, accuracy_score, accuracy_scores_grid = degree_lambda_grid_search(y,tX,[22],0.8,method_flag,degrees,lambdas)
print(best_degree)
print(best_lambda)
print(accuracy_score)
print(accuracy_scores_grid)

### Ridge regression

In [ ]:
method_flag = 4
degrees = np.asarray(range(1,10))
lambdas = np.logspace(-10,0,11)
best_degree, best_lambda, accuracy_score, accuracy_scores_grid = degree_lambda_grid_search(y,tX,[22],0.8,method_flag,degrees,lambdas)
print(best_degree)
print(best_lambda)
print(accuracy_score)
print(accuracy_scores_grid)

### Logistic regression

In [ ]:
method_flag = 5
degrees = np.asarray(range(1,4))
lambdas = np.logspace(-7,0,8)
best_degree, best_lambda, accuracy_score, accuracy_scores_grid = degree_lambda_grid_search(y,tX,[22],0.8,method_flag,degrees,lambdas)
print(best_degree)
print(best_lambda)
print(accuracy_score)
print(accuracy_scores_grid)

### Regularized logistic regression 

In [ ]:
method_flag = 5
degrees = np.asarray(range(1,4))
lambdas = np.logspace(-7,0,8)
best_degree, best_lambda, accuracy_score, accuracy_scores_grid = degree_lambda_grid_search(y,tX,[22],0.8,method_flag,degrees,lambdas)
print(best_degree)
print(best_lambda)
print(accuracy_score)
print(accuracy_scores_grid)

## Ridge regression

In [ ]:
best_lambda = 1e-5
lambda_ = best_lambda
weights, loss = ridge_regression(y_train,x_train,lambda_)

## Least squares

In [ ]:
# weights = least_squares(y_train,x_train)

## Test crossvalidation

### Least squares GD

In [ ]:
method_flag = 1
k_fold = 2
degree = 5
lambda_ = 1e-5
gamma = 2.2e-6
max_iters = 2000 
losses_tr, losses_va, accuracy_scores = k_fold_cross_validation(y,tX,cat_cols,method_flag,k_fold,degree,lambda_,gamma,max_iters)
print(losses_tr)
print(losses_va)
print(accuracy_scores)

### Least squares SGD

In [ ]:
method_flag = 2
k_fold = 2
degree = 5
lambda_ = 1e-5
gamma = 2.2e-6
max_iters = 2000 
losses_tr, losses_va, accuracy_scores = k_fold_cross_validation(y,tX,cat_cols,method_flag,k_fold,degree,lambda_,gamma,max_iters)
print(losses_tr)
print(losses_va)
print(accuracy_scores)

### Least squares

In [ ]:
method_flag = 3
k_fold = 2
degree = 5
lambda_ = 1e-5
gamma = 2.2e-6
max_iters = 2000 
losses_tr, losses_va, accuracy_scores = k_fold_cross_validation(y,tX,cat_cols,method_flag,k_fold,degree,lambda_,gamma,max_iters)
print(losses_tr)
print(losses_va)
print(accuracy_scores)

### Ridge regression

In [ ]:
method_flag = 4
k_fold = 2
degree = 5
lambda_ = 1e-5
gamma = 2.2e-6
max_iters = 2000 
losses_tr, losses_va, accuracy_scores = k_fold_cross_validation(y,tX,cat_cols,method_flag,k_fold,degree,lambda_,gamma,max_iters)
print(losses_tr)
print(losses_va)
print(accuracy_scores)


### Logistic regression

In [ ]:
method_flag = 5
k_fold = 5
degree = 3
lambda_ = 1e-5
gamma = 2.2e-6
max_iters = 2000 
losses_tr, losses_va, accuracy_scores = k_fold_cross_validation(y,tX,cat_cols,method_flag,k_fold,degree,lambda_,gamma,max_iters)
print(losses_tr)
print(losses_va)
print(accuracy_scores)

### Regularized logistic regression

In [ ]:
method_flag = 6
k_fold = 2
degree = 5
lambda_ = 1e-5
gamma = 2.2e-6
max_iters = 2000 
losses_tr, losses_va, accuracy_scores = k_fold_cross_validation(y,tX,cat_cols,method_flag,k_fold,degree,lambda_,gamma,max_iters)
print(losses_tr)
print(losses_va)
print(accuracy_scores)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
y_pred = predict_labels(weights, x_val)
y_val = relabel_y_negative(y_val)

In [ ]:
# Just to check
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, recall_score, precision_score
print(confusion_matrix(y_val,y_pred))
print(accuracy_score(y_val,y_pred))
print(f1_score(y_val,y_pred))
print(recall_score(y_val,y_pred))
print(precision_score(y_val,y_pred))

In [ ]:
get_label_y_counts(y_pred)

In [ ]:
OUTPUT_PATH = '../results/ridge_reg_submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, x_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)